# Introduction

This repository uses an implementation of the [Deep Deterministic Policy Gradients (DDPG)](https://arxiv.org/abs/1509.02971) algorithm. The implementation heavily borrows from the benchmark implementation in the Deep Reinforcement Learning Nanodegree Program. 


# Deep Deterministic Policy Gradients Algorithm

## Elements

### Actor
* The actor $\mu:S \mapsto A$ returns the action $a_t \in A$ given the state $s_t \in S$.
* In the DDPG algorithm, the actor is approximated by the function $\mu:S \times \Theta_\mu \mapsto A$ where
   * $\Theta_\mu$ is set of parameters of the approximation, in this case the weights of the approximating neural network.
* The DDPG algorithm uses 2 neural networks for training purposes:
   * $\theta_\mu \in \Theta_\mu$: local neural network weights. These are updated at every training step using policy gradients.
   * $\hat{\theta}_\mu \in \Theta_\mu$: target neural network weights, these are updated at a slower rate to improve the stability of training.

### Critic
* The critic is the approximation to the Q-Function and is denoted by $Q:S \times A \times \Theta_{q} \mapsto \mathbb{R}$ where
  * $\Theta_\mu$ is the set of parameters of the neural network used in the approximation.
* The DDPG algorithm uses 2 neural networks to for training purposes, hence there are 2 sets of parameters:
  * $\theta_{q} \in \Theta_q$ is the set of parameters used by the local neural network.
  * $\hat{\theta}_{q} \in \Theta_q$ is the set of parameters used by the target neural network and are updated at a lower rate to improve the stability of the training. 

### Noise Process and Randomizations in Actions
* The DDPG algorithm uses noise to induce exploration during training. The noise is generated by an [Ornstein - Uhlenbeck process](https://en.wikipedia.org/wiki/Ornstein%E2%80%93Uhlenbeck_process) denoted by $N(\mu, \theta,\sigma)$ where 
  * $\mu$ is the mean of the noise
  * $\sigma$ is the volatility
  * $\theta$ is the decay-rate.
* The DDPG algorithm computes the action values as follows
  \begin{align*}
   a_t     &= \mu(s_t, \theta_\mu) + \epsilon_t N(\mu, \theta, \sigma)\\
   e_{t+1} &= \gamma_\epsilon \epsilon_t 
  \end{align*}
  where the randomness rate,$\epsilon_t$, is between [0,1] and $\gamma_\epsilon \in (0,1)$ is the decay rate of $\epsilon_t$ and the initial randomness rate $\epsilon_0$. 
* Each element of the action vector is clipped so that they are between -1 and 1:
  \begin{align*}
   a_t[k] \leftarrow \max(\min(1, a_t[k]), -1)
  \end{align*}

### The Experience Replay Buffer 
* DDPG stores the experience $(s_t, a_t, r_{t+1}, s_{t+1})$ where 
  * $s_t$ is the state at time $t$
  * $s_{t+1}$ is the state at time $t+1$
  * $a_t$ is the action at time $t$
  * $r_{t+1}$ is the reward obtained at time $t+1$, 
in a buffer of fixed length $N_B$.
* The stored experiences are used in neural network training.  The experiences are used in batches of size $N_s$. 

### Critic and Actor Training
* At every $T$ time steps, the local critic neural network is trained as follows:
     1. For $K$ update steps repeat:
        1. Select a batch of size $N_s$ experiences randomly from the replay buffer.
        2. For each experience , compute:
           \begin{align*}
           y_i &= r_{t+1,i} + \gamma Q(s_{t+1,i}, \mu(s_{t+1,i},\hat{\theta}_\mu),\hat{\theta}_q)\;\; \forall i \in \{1, \dots, N_s\}
           \end{align*}
           where $\gamma$ is the discount factor.
        3. Update $\theta_q$ as follows:
           \begin{align*}
           \theta_q &\leftarrow \theta_q + \frac{\alpha_q}{N_s} \sum_{i=1}^{N_s}(y_i- Q(s_{t,i},a_{t,i}, \theta_q))\nabla_{\theta_q} Q(s_{t,i},a_{t,i}, \theta_q)
           \end{align*}
           to minimize the objective function
           \begin{align*}
            L = \frac{1}{N_s}\sum_{i =1}^{N_s} (y_i- Q(s_{t,i},a_{t,i}, \theta_q))^2 
           \end{align*} 
           where $\alpha_q$ is the critic learning rate.       
* The local actor neural network weights are updated using a policy gradient approach.
* At every $T$ time steps, the local critic neural network is trained as follows:
     1. For $K$ update steps repeat:
        1. Select a batch of size $N_s$ experiences randomly from the replay buffer.
        2. Compute the policy gradient and update the weights.
        \begin{align*}
            \nabla J_{\theta_{\mu}} &= \frac{1}{N_s}\sum_{i=1}^{N_s}\nabla Q_{a}(s_{t,i},a_{t,i}, \hat{\theta}_q)\nabla_{\theta_\mu}\mu(s_{t,i}, \theta_\mu) \\
            \theta_\mu &\leftarrow \theta_\mu + \alpha_\mu \nabla J_{\theta_{\mu}}
        \end{align*}
        where $\alpha_\mu$ is the learning rate for the actor. 

### Continuous Target Actor and Critic Update
* The target actor and critic neural network weights are updated gradually at every time $T$ time steps and every time the local counterparts are updated according the following rules: 
 \begin{align*}
   \hat{\theta}_q &\leftarrow  (1-\tau)\hat{\theta}_q + \tau\theta_q \\
   \hat{\theta}_{\mu} &\leftarrow  (1-\tau)\hat{\theta}_{\mu} + \tau\theta_{\mu}
  \end{align*}
where $\tau$ is the soft update parameter.

# Overall Algorithm
0. Initialize $\theta_\mu$ and $\theta_q$ randomly. 
   * Set $\hat{\theta}_\mu = \hat{\theta}$ and $\hat{\theta}_q = \theta_q$. 
   * Set the parameters $\tau$,$\gamma$, $\alpha_\mu$, $\alpha_q$, $N_B$, $N_s$, $\mu$, $\theta$, $\sigma$, $\epsilon_0$, $\gamma_\epsilon$, $T$, $K$  
   
1. for $episode =1, \;\; M$ do
   1. Initialize random process $N(\mu,\theta,\sigma)$.
   2. Receive initial state $s_1$
   3. for $t =1, \;t_{final}$ do
      1. compute $a_t$ as  
          \begin{align*}
            a_t     &= \mu(s_t, \theta_\mu) + \epsilon_t N(\mu, \theta, \sigma)\\
            a_t[k]  &\leftarrow \max(\min(1, a_t[k]), -1)\\
            e_{t+1} &= \gamma_\epsilon \epsilon_t 
          \end{align*}
      2. Execute $a_t$, receive $r_{t=1}$ and transition to new state $s_{t+1}$.
      3. Store experience $(s_t, a_t, r_{t+1}, s_{t+1})$ in the experience replay buffer.
      4. If $t\bmod T = 0$ and the number of experiences stored in the replay buffer is greater than $N_s$
         1. for $u = 1,\; K$ do
            1. Obtain a batch of experiences from the replay buffer of size $N_s$.
            2. Update the critic local network weights. 
                 * For each experience , compute:
                   \begin{align*}
                     y_i &= r_{t+1,i} + \gamma Q(s_{t+1,i}, \mu(s_{t+1,i},\hat{\theta}_\mu),\hat{\theta}_q)\;\; \forall i \in \{1, \dots, N_s\}
                    \end{align*}
                 * Update $\theta_q$ as follows:
                   \begin{align*}
                      \theta_q &\leftarrow \theta_q + \frac{\alpha_q}{N_s} \sum_{i=1}^{N_s}(y_i- Q(s_{t,i},a_{t,i}, \theta_q))\nabla_{\theta_q} Q(s_{t,i},a_{t,i}, \theta_q)
                    \end{align*}
            3. Update the actor local network weights using the policy gradient:   
                \begin{align*}
                   \nabla J_{\theta_{\mu}} &= \frac{1}{N_s}\sum_{i=1}^{N_s}\nabla Q_{a}(s_{t,i},a_{t,i}, \hat{\theta}_q)\nabla_{\theta_\mu}\mu(s_{t,i}, \theta_\mu) \\
                   \theta_\mu &\leftarrow \theta_\mu + \alpha_\mu \nabla J_{\theta_{\mu}}
                \end{align*}
            4. Soft update the critic target and actor target neural network weights.   
                \begin{align*}
                   \hat{\theta}_q &\leftarrow  (1-\tau)\hat{\theta}_q + \tau\theta_q \\
                   \hat{\theta}_{\mu} &\leftarrow  (1-\tau)\hat{\theta}_{\mu} + \tau\theta_{\mu}
                \end{align*} 


# Implementation Details
## Neural Network Structure

### Actor Target and Local Neural Networks

* The structure is as follows for both the actor local and actor target neural networks:
    1. Layer: Input Layer(input_dim = size of the state vector, output_dim = 256)
    2. Normalization: Batch normalization(input_dim = 256, output_dim = 256)
    3. Activation: Relu Activation
    4. Layer: Hidden Layer(input_dim = 256, output_dim = 256)
    5. Activation: Relu Activation
    6. Layer: Hidden Layer(input_dim = 256, output_dim = size of the action vector)
    7. Activation: Tanh
    
### Critic Target and Local Neural Networks

* The structure is as follows for both the 
    1. Layer: Input Layer (input_dim = size of the state vector,output_dim = 256)
    2. Normalization: Batch Normalization
    3. Activation: Relu
    4. Layer: Hidden (input_dim = 256 + size of the action vector,  output_dim = 256)
    5. Activation: Relu
    6. Layer: Output Layer(input_dim = 256, output_dim = 1)
    
## Hyperparameters

* Soft update parameter,$\tau$: 0.001
* Discount factor, $\gamma$:0.99
* Learning rate for the actor local training, $\alpha_\mu$:0.00005
* Learning rate for the critic local training, $\alpha_q$:0.00005
* Replay buffer size, $N_B$: 1,000,000
* Training batch size $N_s$: 128
* Ornstein-Uhlenbeck mean, $\mu$: 0.0
* Ornstein-Uhlenbeck decay rate, $\theta$:0.15
* Ornstein-Uhlenbeck volatility, $\sigma$:0.2
* Initial randomness rate, $\epsilon_0$: 1.0
* Randomness rate decay rate, $\gamma_\epsilon$: 0.999
* Frequency of update for the neural networks, $T$: 5
* Number of steps taken at every update, $K$: 10



# Training Results
 The agent achieves an average score of 13 or above in about 113 episodes.
* The progression of the average of the  scores of all agents and the moving average score over 100 episodes is shown in the next figure.
* `Moving Average Score over 100 Episodes` is the moving average score and `Average Score` is the average of the scores at each episode. 
![Training Performance](training_performance.png)


# Future Directions
* The hyper parameters for the algorithm have been chosen by trial and error. Better parameters can be found by using an automated hyperparameter search techniques.
* [Prioritized experienced replay](https://arxiv.org/abs/1511.05952) can improve the performance with less	increase in training time. Here, the training samples are given priorities based on	their TD error.
* Experiment with  other algorithms:
   * [Trust Region Policy Optimization](https://arxiv.org/abs/1502.05477)
   * [D4PG](https://openreview.net/pdf?id=SyZipzbCb):  This algorithm includes prioritized experience replay and is shown to achieve higher performance quicker than DDPG. 